We consider the relations between the cylindrical coordinate variables and the Cartesian coordinate variables.

First of all note that since $x = r\cos\theta$, $y=r\sin\theta$, from which  $x^2 + y^2 = r^2$ and $\displaystyle \tan \theta = \frac{y}{x}$, we have the following:

\begin{alignat*}{2}
\frac{\partial r}{\partial x} &= \cos \theta, &\qquad
\frac{\partial r}{\partial y} &= \sin \theta, \\
\frac{\partial \theta}{\partial x} &= - \frac{\sin \theta}{r}, &
\frac{\partial \theta}{\partial y} &= \frac{\cos\theta}{r}.
\end{alignat*}

Furthermore, since the $z$ coordinate directions in both the cylindrical and the Cartesian systems are coincident, we have:

\begin{gather*}
\frac{\partial r}{\partial z} = 0, \quad \frac{\partial \theta}{\partial z} = 0, \quad \frac{\partial z}{\partial z} = 1,
\end{gather*}
where each of the $\displaystyle \frac{\partial ()}{\partial z}$ is the partial derivative with respect to the $z$ coordinate variable in the Cartesian system. 

In [1]:
import sympy as sym

r, theta, z = sym.symbols('r, theta, z')

delr_delx = sym.cos(theta)

delr_dely = sym.sin(theta)

delr_delz = 0

deltheta_delx = -sym.sin(theta)/r

deltheta_dely = sym.cos(theta)/r

deltheta_delz = 0

delz_delx = 0

delz_dely = 0 

delz_delz = 1

For a generic function, $f(r,\theta,z)$, we have the following relations:

\begin{align*}
\frac{\partial f}{\partial x} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial x} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial x} + \frac{\partial f}{\partial z}\frac{\partial z}{\partial z} = \frac{\partial f}{\partial r}\cos\theta - \frac{\partial f}{\partial \theta}\frac{\sin \theta}{r} + \frac{\partial f}{\partial z}, \\
\frac{\partial f}{\partial y} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial y} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial y} + \frac{\partial f}{\partial z}\frac{\partial z}{\partial z} = \frac{\partial f}{\partial r} \sin\theta + \frac{\partial f}{\partial \theta}\frac{\cos \theta}{r} + \frac{\partial f}{\partial z}, \\
\frac{\partial f}{\partial z} &= \frac{\partial f}{\partial z}
\end{align*}

The $\displaystyle \frac{\partial f}{\partial z}$ remains unchanged in Cartesian and cylindrical coordinates. 

We define the following Python functions to make these derivatives operable on any function of $r$, $\theta$, and $z$:

In [2]:
def del_delx(f):
    return delr_delx*sym.diff(f,r) + deltheta_delx*sym.diff(f,theta) + sym.diff(f,z)*delz_delx

def del_dely(f):
    return delr_dely*sym.diff(f,r) + deltheta_dely*sym.diff(f,theta) + sym.diff(f,z)*delz_dely

def del_delz(f):
    return delr_delz*sym.diff(f,r) + deltheta_delz*sym.diff(f,theta) + sym.diff(f,z)*delz_delz

Define the rotation matrix:

In [3]:
Q = sym.Matrix([[sym.cos(theta), sym.sin(theta), 0],[-sym.sin(theta), sym.cos(theta), 0],[0, 0, 1]])
display(Q)

Matrix([
[ cos(theta), sin(theta), 0],
[-sin(theta), cos(theta), 0],
[          0,          0, 1]])

We set up the stress tensor in the cylindrical coordinate system:

In [4]:
sigmarr = sym.Function(r'\sigma_{rr}')(r,theta,z)
sigmatt = sym.Function(r'\sigma_{\theta\theta}')(r,theta,z)
sigmazz = sym.Function(r'\sigma_{zz}')(r,theta,z)
sigmart = sym.Function(r'\sigma_{r\theta}')(r,theta,z)
sigmazr = sym.Function(r'\sigma_{zr}')(r,theta,z)
sigmatz = sym.Function(r'\sigma_{\theta z}')(r,theta,z)
#display(sigmarr,sigmatt, sigmart)
stress_cyl = sym.Matrix([[sigmarr, sigmart, sigmazr],[sigmart, sigmatt, sigmatz],[sigmazr, sigmatz, sigmazz]])
display(stress_cyl)

Matrix([
[     \sigma_{rr}(r, theta, z),      \sigma_{r\theta}(r, theta, z),       \sigma_{zr}(r, theta, z)],
[\sigma_{r\theta}(r, theta, z), \sigma_{\theta\theta}(r, theta, z), \sigma_{\theta z}(r, theta, z)],
[     \sigma_{zr}(r, theta, z),     \sigma_{\theta z}(r, theta, z),       \sigma_{zz}(r, theta, z)]])

We transform from the cylindrical to the Cartesian coordinate system using $\displaystyle [\sigma_{\rm Cart}] = [Q]^{\sf T}[\sigma_{\rm cyl}][Q]$:

In [5]:
stress_Cart = (Q.T)*stress_cyl*Q

We extract the stress components in the Cartesian system:

In [6]:
sigmaxx = stress_Cart[0,0]
sigmaxy = stress_Cart[0,1]
sigmazx = stress_Cart[0,2]
sigmayy = stress_Cart[1,1]
sigmayz = stress_Cart[1,2]
sigmazz = stress_Cart[2,2]

We set up the stress equilibrium equations using these stress components in the Cartesian system, but we consider only the lhs terms. Thus, we consider the components of $\nabla \cdot \sigma_{\rm Cart}$, where the divergence operator is of course in the Cartesian system:

In [7]:
lhs1 = del_delx(sigmaxx) + del_dely(sigmaxy) + del_delz(sigmazx)
lhs2 = del_delx(sigmaxy) + del_dely(sigmayy) + del_delz(sigmayz)
lhs3 = del_delx(sigmazx) + del_dely(sigmayz) + del_delz(sigmazz)

We consider these three lhs terms as the components of a vector, equivalently the elements of a column matrix:

In [8]:
div_sigma_Cart = sym.Matrix([lhs1, lhs2, lhs3])

The above vector written in the Cartesian system can be easily transformed to the cylindrical system ($\nabla \cdot \sigma_{\rm cyl}$, where the divergence operator is in the cylindrical system) using $[(\nabla \cdot \sigma_{\rm cyl})] = [Q][(\nabla \cdot \sigma_{\rm Cart})]$:

In [9]:
div_sigma_cyl = Q*div_sigma_Cart

We extract the three components of $\nabla \cdot \sigma_{\rm cyl}$ giving us the lhs of the stress equilibrium equations in the cylindrical coordinate system:

In [10]:
div_sigma_cyl_r = div_sigma_cyl[0].simplify().expand()
div_sigma_cyl_theta = div_sigma_cyl[1].simplify().expand()
div_sigma_cyl_z = div_sigma_cyl[2].simplify().expand()

display(div_sigma_cyl_r, div_sigma_cyl_theta, div_sigma_cyl_z)

Derivative(\sigma_{rr}(r, theta, z), r) + Derivative(\sigma_{zr}(r, theta, z), z) - \sigma_{\theta\theta}(r, theta, z)/r + \sigma_{rr}(r, theta, z)/r + Derivative(\sigma_{r\theta}(r, theta, z), theta)/r

Derivative(\sigma_{\theta z}(r, theta, z), z) + Derivative(\sigma_{r\theta}(r, theta, z), r) + 2*\sigma_{r\theta}(r, theta, z)/r + Derivative(\sigma_{\theta\theta}(r, theta, z), theta)/r

Derivative(\sigma_{zr}(r, theta, z), r) + Derivative(\sigma_{zz}(r, theta, z), z) + \sigma_{zr}(r, theta, z)/r + Derivative(\sigma_{\theta z}(r, theta, z), theta)/r